In [ ]:
# |default_exp langchain_rag

Please reference [this blog post](https://nbdev.fast.ai/blog/posts/2022-11-07-spaces) on how to use this notebook.

## Install dependencies

## Make an app with Gradio

In [1]:
# |export
import gradio as gr
from dotenv import load_dotenv
import os
import time
import getpass

from openai import api_key


In [2]:
#| export
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.llms import ChatMessage


In [3]:
# |export
load_dotenv()
#os.environ['HTTP_PROXY'] = ''
#os.environ['HTTPS_PROXY'] = ''
#os.environ['NO_PROXY'] = 'localhost, 127.0.0.1'
print(os.environ.get('OPENROUTER_API_KEY'))
print(os.environ.get('OPENROUTER_API_URL'))
print(os.environ.get('PINECONE_API_KEY'))
print(os.environ.get('HTTP_PROXY'))
print(os.environ.get('HTTPS_PROXY'))

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
https://openrouter.ai/api/v1
pcsk_4JQiw5_Kg3hyxuJet9Dumt7zfnXySGp2xZHoQqLsuvHNiTVD98JysmAEqxRbsJEu41Ko7P
http://127.0.0.1:20171
http://127.0.0.1:20171


In [28]:
from pinecone import Pinecone, ServerlessSpec, Index
import os
import pinecone

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [29]:
dataset_name = 'quickstart'
if dataset_name not in pc.list_indexes().names():
    pc.create_index(
        dataset_name,
        dimension=1536,
        metric="euclidean",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        )
    ),
pinecone_index = pc.Index(dataset_name)

In [30]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

ModuleNotFoundError: No module named 'llama_index.vector_stores'

In [22]:

# Embed data
data = [
    {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
    {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
    {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
    {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
    {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
]

embeddings = pc.inference.embed(
    model="llama-text-embed-v2",
    inputs=[d['text'] for d in data],
    parameters={
        "input_type": "passage"
    }
)

vectors = []
for d, e in zip(data, embeddings):
    vectors.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

In [23]:
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 5}

In [24]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'euclidean',
 'namespaces': {'ns1': {'vector_count': 5}},
 'total_vector_count': 5,
 'vector_type': 'dense'}

In [5]:
llm = OpenRouter(
    api_key=os.environ.get('OPENROUTER_API_KEY'),
    max_tokens=256,
    context_window=4096,
    model="qwen/qwen2.5-vl-32b-instruct:free"
    # model="deepseek/deepseek-r1:free"
    # model="gryphe/mythomax-l2-13b:free"
)

In [11]:
from openai import OpenAI
from os import getenv

client = OpenAI(
    base_url=os.environ.get('OPENROUTER_API_URL'),
    api_key=os.environ.get('OPENROUTER_API_KEY'),
)

completion = client.chat.completions.create(
    # model="bytedance-research/ui-tars-72b:free",
    model="google/gemini-2.5-pro-exp-03-25:free",
    extra_headers={
        "HTTP-Referer": "binjian.github.io",
        "X-Title": "My Test",
    },
    messages=[
        {
            "role": "user",
            "content": "给我讲个川普和普京的笑话吧."
        },
    ]
)
print(completion.choices[0].message.content)

好的，这是一个关于川普和普京的笑话：

有一天，川普和普京一起在河边钓鱼。

川普用的是最高科技的鱼竿，最贵的鱼饵，旁边还放着一个巨大的冷藏箱，他不停地跟普京吹嘘：“弗拉基米尔，你看我这装备，顶级的！绝对能钓上最大的鱼！没有人比我更懂钓鱼，相信我！”

普京只是默默地坐在那里，用一根看起来很普通的旧鱼竿，时不时还往河里瞟几眼。

过了一会儿，普京一条接一条地钓上大鱼，鱼桶都快满了。而川普那边，浮标动都没动一下。

川普气得脸都红了，抱怨道：“这不公平！这绝对是被操控了！这些鱼肯定是假鱼（Fake Fish）！或者是深水州（Deep State
）派来的鱼！”

普京淡淡地看了他一眼，把一条刚钓上来的大鲈鱼扔进桶里，平静地说：“唐纳德，也许……鱼只是更喜欢那些能安静等待、并且知道在哪里下钩的渔夫？”

---
**温馨提示：** 这只是一个基于公众人物刻板印象的笑话，目的是为了娱乐，并不代表他们真实的情况或关系。希望你觉得好笑！


In [35]:
llm = OpenRouter(
    api_key=os.environ.get('OPENROUTER_API_KEY'),
    max_tokens=256,
    context_window=4096,
    model="google/gemma-3-12b-it:free"
    # model="google/gemini-2.5-pro-exp-03-25:free"
    # model="deepseek/deepseek-r1:free"
    # model="gryphe/mythomax-l2-13b:free"
)
message = ChatMessage(role="user", content="Tell me a joke.")
resp = llm.chat([message])
print(resp)

assistant: 

Why don't scientists trust atoms?

Because they make up everything! 😂



Hope that gave you a chuckle!


In [38]:
message = ChatMessage(role="user", content="请讲述一个250字的科幻小说故事")
resp = llm.stream_chat([message])
for r in resp:
    print(r.delta, end='', flush=True)



## 遗忘之海

**第一章：锈蚀的信号**

星历3247年，我，艾瑞斯·科尔，是一名星际考古学家，专门研究失落文明的遗迹。我的飞船“探寻者号”正漂浮在遗忘之海边缘，一个被星际联盟划定为禁区，因为这里充斥着无法解释的能量波动和令人不安的幻觉。

遗忘之海，曾经是繁星点点的星域，如今却只剩下一片深邃的黑色，仿佛宇宙本身被撕裂了一道口子。几百年前，一个名为“阿卡迪亚”的文明突然消失于此，留下的只有无尽的黑暗和令人毛骨悚然的传说。

我之所以来到这里，是因为收到了一段微弱的信号，一段来自阿卡迪亚文明的求救信号。联盟认为这只是宇宙背景噪音，但我的直觉告诉我，这并非如此。阿卡迪亚文明以其高度发达的科技和对精神领域的探索而闻名，如果他们真的遭遇了什么，那将是足以改变星际历史的事件。

“探寻

In [40]:
resp = llm.complete("Tell me a joke in Feynman style.")
print(resp)

Okay, here's a joke, explained in a Feynman-esque way. It's not *just* the joke, but the process of getting to the joke, and then a little bit of why it's funny, all laid out like Richard Feynman might have approached it.

**(Deep breath, adjusts glasses, leans forward slightly)**

Alright, so you know how we talk about probability, right? Like, flipping a coin.  It's supposed to be 50/50, heads or tails.  Simple, right?  But *really*, it's a whole mess of physics going on. You've got the initial force you put on it, the angle, the air resistance, the spin... all these things influencing where it lands.  It's chaotic!  You can't *perfectly* predict it.  That's why we use probability – it's a way of dealing with the fact that we don't know all the details.

Now, imagine you're flipping a coin, and you flip it, and it lands on heads.  And you flip it again, and it's heads.  And again... heads.  Five heads in a row.  You


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from os import getenv
from dotenv import load_dotenv
load_dotenv()

template = """Question: {question}
Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm = ChatOpenAI(
  openai_api_key=getenv("OPENROUTER_API_KEY"),
  openai_api_base=getenv("OPENROUTER_BASE_URL"),
  model_name="<model_name>",
  model_kwargs={
    "headers": {
      "HTTP-Referer": getenv("YOUR_SITE_URL"),
      "X-Title": getenv("YOUR_SITE_NAME"),
    }
  },
)
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"
print(llm_chain.run(question))

In [8]:
message = ChatMessage(role="user", content="给我讲个少林寺笑话吧.")
resp = llm.chat(messages=[message])
print(resp)

assistant: 好的，我给你讲一个关于少林寺的笑话：

---

有一天，少林寺的方丈召集众弟子，说：“今天我要考考你们的智慧。我给你们每人一个木鱼，谁能在最短的时间内让木鱼发出声音，谁就能得到奖励。”

众弟子一听，纷纷开始想办法。有的弟子把木鱼放在石头上敲，有的弟子用木棍敲，还有的弟子甚至把木鱼扔到地上摔。

这时，一个小和尚站了出来，他拿起木鱼，放在耳边，认真地听了一会儿，然后对大家说：“我听到声音了！”

众弟子都愣住了，纷纷问：“你怎么听到的？”

小和尚得意地说：“很简单啊，我用耳朵听的！”

方丈听了，哈哈大笑，说：“你这个小和尚，真是聪明！”

---

这个笑话虽然简单，但寓意深刻，提醒我们有时候解决问题的方法可能并不复杂，关键在于换个角度思考。希望你喜欢！ 😄


In [ ]:
#| export
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
#| export
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)


In [ ]:
#| export
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
import validators

In [ ]:
#| export
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
#| export
# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
#| export
def answer(message, history, system_prompt, tokens):
    files = []
    file_names = []
    for msg in history:
        if msg["role"] == "user" and isinstance(msg["content"], tuple):
            files.append(msg["content"][0])
            file_names.append(msg["content"][0].split("/")[-1])
    for file in message["files"]:
        files.append(file)
        file_names.append(file.split("/")[-1])

    #if message["text"]:
    #    content = message["text"]
    #else:
    #    content = system_prompt
    # content = message
    # question = system_prompt
    # response = f"Content: {content}\nQuestion: {question}\n"
    # len = min(len(response),int(response_len))

    user_input = f"Question: {system_prompt}\n Website: {message['text']}\n File:\n{'\n'.join(file_names)}"

    if validators.url(message['text']):
        loader = WebBaseLoader(
            # web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
            web_paths=(message['text'],),
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    class_=("post-content", "post-title", "post-header")
                )
            ),
        )
        docs = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        all_splits = text_splitter.split_documents(docs)
        # Index chunks
        _ = vector_store.add_documents(documents=all_splits)

        # # # Compile application and test
        # graph_builder_i = StateGraph(State).add_sequence([retrieve, generate])
        # graph_builder_i.add_edge(START, "retrieve")
        # graph_i = graph_builder_i.compile()
        reply = graph.invoke({"question": system_prompt})
        response_i = reply["answer"]
    elif files:
        f = files[-1]
        f_name = file_names[-1]多久维护一次产品?
        response_i = f"File: {f_name}\n"

    # response_i = user_input
    for i in range(min(len(response_i), int(tokens))):
        time.sleep(0.05)
        yield response_i[: i + 1]

In [ ]:
# |export
demo = gr.ChatInterface(
    answer,
    type="messages",
    title="智能问答RAG",
    description="输入一个网址，查询或询问其中的内容。",
    textbox=gr.MultimodalTextbox(value="https://lilianweng.github.io/posts/2023-06-23-agent/",
                                 file_count="multiple",
                                 file_types=["image", ".pdf", ".txt"],
                                 sources=["upload", "microphone"]),
    additional_inputs=[
        gr.Textbox("What is Task Decomposition?", label="你的问题在此输入！"),
        gr.Slider(10,400,value=300,label="回答长度")
    ],
    multimodal=True,
)
demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [ ]:
# this is only necessary in a notebook
demo.close()

Closing server running on port: 7865


## Create a `requirements.txt` file

In [ ]:
%%writefile ../requirements.txt
fastcore

Writing ../requirements.txt


## Convert this notebook into a Gradio app

In [ ]:
# from nbdev.export import nb_export
# nb_export('01_gradio.ipynb', lib_path='.', name='gradio')

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

<div>
<link rel="stylesheet" href="https://gradio.s3-us-west-2.amazonaws.com/2.6.5/static/bundle.css">
<div id="target"></div>
<script src="https://gradio.s3-us-west-2.amazonaws.com/2.6.5/static/bundle.js"></script>
<script>
launchGradioFromSpaces("abidlabs/question-answering", "#target")
</script>
</div>